In [9]:
import time
import csv
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

from src.Clients.BlockchainApiClient import Client as BlockchainAPIClient
from src.Clients.GoogleTrendsClient import Client as GoogleTrendsClient

# Pobieranie danych

In [10]:
START_YEAR = 2018
END_YEAR = 2025
OUTPUT_CSV = "assets/crypto-data-new.csv"
file_exists = os.path.exists(OUTPUT_CSV)
headers_written = file_exists

## Yahoo Finance

In [11]:
yahooClient = yf.Ticker("BTC-USD")

## Google Trends

In [12]:
keywords = ["Bitcoin", "BTC"]
trendsClient = GoogleTrendsClient(keywords)

## Blockchain.com

In [13]:
blockchainClient = BlockchainAPIClient()

# Tworzenie CSV

In [14]:
headers_written = False

for year in range(START_YEAR, END_YEAR + 1):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    print(f"Pobieranie danych dla roku {year}...")

    # Fetch
    btc_history = yahooClient.history(start=start_date, end=f"{year+1}-01-01")
    # trends_data = trendsClient.get_chart_data(timeframe=f"{start_date} {end_date}")
    blockchain_metrics = [
        "avg-block-size",
        "n-transactions-per-block",
        "n-payments-per-block",
        "transactions-per-second", 
        "blocks-size",
        "hash-rate",
        "difficulty",
    ]

    blockchain_data = {metric: blockchainClient.get_chart_data(metric, f"{year - 1}-12-31", '2year') for metric in blockchain_metrics}

    year_data = {}

    for date in pd.date_range(start=start_date, end=end_date):
        date_str = date.strftime("%Y-%m-%d")
        year_data[date_str] = {}

    for idx, row in btc_history.iterrows():
        row_date_str = idx.strftime("%Y-%m-%d")
        if row_date_str in year_data:
            year_data[row_date_str].update(row.to_dict())

    # if not trends_data.empty:
    #     for idx, row in trends_data.iterrows():
    #         row_date_str = idx.strftime("%Y-%m-%d")
    #         if row_date_str in year_data:
    #             year_data[row_date_str].update(row.to_dict())

    for metric, data in blockchain_data.items():
        for point in data["values"]:
            # Konwersja x (sekundy) na datę w strefie Europe/Warsaw
            converted_date = pd.to_datetime(point["x"], unit="s", utc=True) \
                                .tz_convert("Europe/Warsaw") \
                                .strftime("%Y-%m-%d")
            if converted_date in year_data:
                year_data[converted_date][metric] = point["y"]
    
    all_columns = ['date'] + list(year_data[start_date].keys())

    if not headers_written:
        with open(OUTPUT_CSV, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(all_columns)
        headers_written = True

    # Zapisujemy dane do pliku CSV
    with open(OUTPUT_CSV, mode="a", newline="") as file:
        writer = csv.writer(file)
        for date, row in year_data.items():
            # Zapewniamy, że dane są w tej samej kolejności co nagłówki
            row_data = [date] + [row.get(col, '') for col in all_columns[1:]]
            writer.writerow(row_data)

    # Przerwa między żądaniami, by nie przeciążyć API
    time.sleep(10)

print(f"Dane zapisano do {OUTPUT_CSV}")

Pobieranie danych dla roku 2018...
Pobieranie danych dla roku 2019...
Pobieranie danych dla roku 2020...
Pobieranie danych dla roku 2021...
Pobieranie danych dla roku 2022...
Pobieranie danych dla roku 2023...
Pobieranie danych dla roku 2024...
Pobieranie danych dla roku 2025...
Dane zapisano do assets/crypto-data-new.csv
